In [ ]:
from torch import nn
from torchsummary import summary
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torchvision import datasets, transforms as T
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
device = 'cuda'
batch_size = 128
noise_dim = 64

lr = 0.0002
beta_1 = 0.5
beta_2 = 0.99

epochs = 20

In [ ]:
train_augs = T.Compose([
    T.RandomRotation((-20, +20)),
    T.ToTensor()  # (h, w, c) -> (c, h, w)
])
trainset = datasets.MNIST("MNIST/", download=True,
                          train=True, transform=train_augs)


In [ ]:
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
print(f"number of batches = {len(trainloader)}")

dataiter = iter(trainloader)
images, _ = dataiter._next_data()
print(images.shape)

In [ ]:
def show_tensor_images(tensor_img, num_images=16, size=(1, 28, 28)):
    unflat_img = tensor_img.detach().cpu()
    img_grid = make_grid(unflat_img[:num_images], nrow=4)
    plt.imshow(img_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
show_tensor_images(images,num_images=16)

## Discriminator

In [ ]:

def get_disc_block(in_channels, out_channels, kernel_size, stride):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2)
    )



In [ ]:
class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()
        self.block_1 = get_disc_block(1, 16, (3, 3), 2)
        self.block_2 = get_disc_block(16, 32, (5, 5), 2)
        self.block_3 = get_disc_block(32, 64, (5, 5), 2)

        self.flatten = nn.Flatten()
        self.linear = nn.Linear(in_features=64, out_features=1)

    def forward(self, images):
        x1 = self.block_1(images)
        x2 = self.block_2(x1)
        x3 = self.block_3(x2)

        x4 = self.flatten(x3)
        x5 = self.linear(x4)

        return x5

In [ ]:
D = Discriminator()
# D.to(device=device)

summary(D, input_size=(1, 28, 28))

## Generator

In [ ]:
def get_gen_block(in_channels, out_channels, kernel_size, stride, final_block=False):
    if final_block == True:
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride),
            nn.Tanh()
        )
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

In [ ]:

class Generator(nn.Module):

    def __init__(self, noise_dim):
        super(Generator, self).__init__()

        self.noise_dim = noise_dim
        self.block_1 = get_gen_block(noise_dim, 256, (3, 3), 2)
        self.block_2 = get_gen_block(256, 128, (4, 4), 1)
        self.block_3 = get_gen_block(128, 64, (3, 3), 2)

        self.block_4 = get_gen_block(64, 1, (4, 4), 2, final_block=True)

    def forward(self, r_noise_vec):
        # (batch_size, noise_dim) -> (batch_size, noise_dim, 1, 1)
        x = r_noise_vec.view(-1, self.noise_dim, 1, 1)

        x1 = self.block_1(x)
        x2 = self.block_2(x1)
        x3 = self.block_3(x2)
        x4 = self.block_4(x3)

        return x4


In [ ]:
G = Generator(noise_dim)
# G.to(device)

summary(G, input_size=(1, noise_dim))

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight, 0.0, 0.02)
        nn.init.constant_(m.bias, 0)


D = D.apply(weights_init)
G = G.apply(weights_init)

In [ ]:

def real_loss(disc_pred):
    criterion = nn.BCEWithLogitsLoss()
    ground_truth = torch.ones_like(disc_pred)
    loss = criterion(disc_pred, ground_truth)
    return loss


def fake_loss(disc_pred):
    criterion = nn.BCEWithLogitsLoss()
    ground_truth = torch.zeros_like(disc_pred)
    loss = criterion(disc_pred, ground_truth)
    return loss


In [ ]:
D_opt = torch.optim.Adam(D.parameters(), lr=lr, betas=(beta_1, beta_2))
G_opt = torch.optim.Adam(G.parameters(), lr=lr, betas=(beta_1, beta_2))

## Training Loop

In [ ]:
for i in range(epochs):

    total_d_loss = 0.0
    total_g_loss = 0.0

    for real_img, _ in tqdm(trainloader):
        # real_img = real_img.to(device)
        noise = torch.randn(batch_size, noise_dim)

        # find loss and update weights for D
        D_opt.zero_grad()
        fake_img = G(noise)

        D_pred = D(fake_img)
        D_fake_loss = fake_loss(D_pred)

        D_pred = D(real_img)
        D_real_loss = real_loss(D_pred)

        D_loss = (D_fake_loss + D_real_loss)/2
        total_d_loss += D_loss.item()

        D_loss.backward()
        D_opt.step()

        # find loss and update weights for G
        G_opt.zero_grad()
        noise = torch.randn(batch_size, noise_dim)

        fake_img = G(noise)
        D_pred = D(fake_img)

        G_loss = real_loss(D_pred)
        total_g_loss += G_loss.item()

        G_loss.backward()
        G_opt.step()

    print(f"total_d_loss = {total_d_loss}")
    print(f"total_g_loss = {total_g_loss}")
    avg_d_loss = total_d_loss / len(trainloader)
    avg_g_loss = total_g_loss / len(trainloader)

    print(f"Epoch : {i + 1} | D_loss : {avg_d_loss} | G_loss = {avg_g_loss}")
    show_tensor_images(fake_img)
